In [ ]:
pip install -q yahoo_fin

In [2]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml@v0.0.6#egg=runml

In [4]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

Run following if like to save results in drive

In [5]:
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [6]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accuracy']/stop_loss - (1-df['Accuracy'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
pending = ['PAR']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP', 'AYX',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HIMS','HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JKS', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PERI', 'PANW',  'PAYX', 'PD', 'PLUG', 'PI', 'PINS', 'PUBM',
           'RBLX', 'RMBS', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df  = pipeline.runModelCombined(tickers, 'ipos-4a', mod, False, loss=lossfn)[0]

In [ ]:
addAlloc(df, 0.10, 1)
df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
48,OLED,RROnly,91.315339,0.946809,843.63,1046.65,1890.28,173.14,189.000000,0.09,8.877069
71,U,RROnly,23.738379,0.936047,526.37,934.37,1460.74,21.67,23.959999,0.11,8.779070
0,ABNB,RROnly,85.057767,0.918750,764.69,1088.55,1853.24,145.66,161.050003,0.11,8.448864
77,Z,RROnly,29.174104,0.909574,547.36,738.82,1286.18,44.10,38.340000,-0.13,8.400164
43,MXL,RROnly,15.293257,0.930851,268.44,500.22,768.66,19.45,20.889999,0.07,8.320669
21,ENPH,RROnly,44.374956,0.904255,2066.78,2174.39,4241.17,114.32,129.119995,0.13,8.306056
50,PANW,RROnly,79.164817,0.888298,1531.39,835.22,2366.61,317.85,257.279999,-0.19,8.295073
65,TEAM,RROnly,122.324382,0.904255,2252.16,1902.33,4154.49,179.72,200.720001,0.12,8.244681
70,TWLO,RROnly,48.288491,0.914894,1769.71,2408.73,4178.44,62.08,56.209999,-0.09,8.203310
69,TTD,RROnly,33.805407,0.909574,809.62,656.35,1465.96,94.78,85.120003,-0.10,8.191489


In [ ]:
findata.EPOCHS=200
later = [ ]
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df1, res = pipeline.runModelCombined(tickers1, 'etf-5b', mod, True)


In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

(0.9202127659574468,
 0.9229203567322334,
 0.09075827015321843,
 0.004420030125699004,
 0.01842253193096099)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
10,IPO,RROnly,24.522572,0.968085,232.29,341.32,573.61,39.66,42.419998,0.07,9.224924
2,ARKW,RROnly,38.331096,0.941489,768.19,699.70,1467.89,78.19,85.050003,0.09,8.764775
24,SMOG,RROnly,76.522493,0.946809,751.10,803.64,1554.75,102.53,109.110001,0.06,8.581560
25,SPY,RROnly,286.481888,0.952128,1456.58,966.14,2422.73,529.45,510.329987,-0.04,8.324468
26,TDIV,RROnly,37.451612,0.941489,194.56,103.70,298.26,72.06,68.480003,-0.05,8.244681
44,XME,RROnly,20.195892,0.930851,355.42,211.72,567.14,64.00,59.959999,-0.06,8.156028
0,ARKF,RROnly,14.224096,0.893617,183.92,214.90,398.82,28.10,31.049999,0.11,7.969052
12,IXN,RROnly,36.459918,0.930851,284.75,145.78,430.54,75.90,72.150002,-0.05,7.925532
1,ARKK,RROnly,31.170188,0.920213,454.41,715.16,1169.57,45.45,42.689999,-0.06,7.872340
18,MGK,RROnly,153.788720,0.936170,1106.66,524.59,1631.25,291.15,280.029999,-0.04,7.765957


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers3 = [ 'AIRI', 'AEXAF', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS', 'AUGX', 'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CLOV', 'CRCT', 'CXM', 'CZOO',
            'DLO',  'DM', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MRDB', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PTON', 'REAL', 'REI', 'RIG', 'RKLB', 'RUM', 'RVYL',
            'SABR', 'SAVE', 'SFIX', 'SLE', 'SMWB', 'SOUN', 'SST', 'STEM',
            'TDW', 'TRUP', 'TELL',
            'VTEX', 'YEXT', 'WKME']
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df3 = pipeline.runModelCombined(tickers3, 'vols-4b', mod, False)[0]

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
50,OPEN,RROnly,1.360753,0.946237,115.46,191.35,306.81,2.13,2.790000,0.31,5.392667
25,EVGO,RROnly,2.013745,0.939024,75.77,129.85,205.62,2.14,2.780000,0.30,5.320421
69,STEM,RROnly,1.773574,0.941176,198.98,230.13,429.11,1.30,1.610000,0.24,5.291234
14,BLDE,RROnly,2.316142,0.930851,70.32,96.42,166.74,3.21,4.230000,0.32,5.259504
6,APPS,RROnly,3.006594,0.920213,502.58,518.08,1020.66,2.14,2.960000,0.38,5.203050
41,LUMN,RROnly,1.175467,0.904255,49.27,75.08,124.34,1.27,1.880000,0.48,5.119681
49,ONDS,RROnly,0.505266,0.876543,37.59,54.85,92.44,0.75,1.450000,0.94,5.024800
7,AUGX,RROnly,1.237636,0.890411,17.23,37.54,54.77,1.14,1.680000,0.47,5.004543
47,NNOX,RROnly,6.026901,0.881356,303.56,315.20,618.76,8.43,13.010000,0.54,4.964735
66,SMWB,RROnly,4.831706,0.907801,42.22,88.88,131.10,8.09,9.710000,0.20,4.879015


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
mod4 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df4 = pipeline.runModelCombined(tickers4, 'mcap-5b', mod4, True)[0]

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
25,META,RROnly,94.987541,0.893617,2110.66,1313.86,3424.52,478.22,406.380005,-0.15,5.673759
18,GOOGL,RROnly,70.014863,0.925532,771.72,475.21,1246.93,174.99,163.520004,-0.07,5.547112
4,AMZN,RROnly,84.424588,0.930851,835.48,688.94,1524.43,180.75,169.279999,-0.06,5.496454
39,UPS,RROnly,91.211122,0.909574,810.08,699.43,1509.51,138.66,151.699997,0.09,5.492232
6,BA,RROnly,119.704902,0.914894,1222.35,1429.02,2651.37,174.52,160.580002,-0.08,5.471125
2,ADBE,RROnly,283.011501,0.957447,2764.33,3230.06,5994.39,475.43,487.519989,0.03,5.420466
14,DE,RROnly,147.867772,0.888298,2221.55,1377.75,3599.31,374.96,421.209991,0.12,5.414134
23,ISRG,RROnly,186.976383,0.925532,1811.57,1350.31,3161.88,404.85,378.769989,-0.06,5.369807
37,TXN,RROnly,112.786096,0.920213,672.19,693.90,1366.09,199.18,187.199997,-0.06,5.243161
38,UNH,RROnly,273.524197,0.904255,1732.47,1316.58,3049.04,508.17,472.570007,-0.07,5.091185


In [ ]:
findata.EPOCHS=200
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX', 'DBOEY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']
mod5 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df5 = pipeline.runModelCombined(tickers5, 'gstock-4b', mod5, False)[0]


In [ ]:
addAlloc(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
48,PROSF,RROnly,28.607553,0.941489,428.52,590.52,1019.04,37.35,43.869999,0.17,6.036564
47,POAHF,RROnly,45.264391,0.925532,311.31,572.67,883.98,54.57,61.169998,0.12,5.369807
23,FANUY,RROnly,12.323899,0.909574,78.89,96.98,175.87,13.97,16.170000,0.16,5.366641
45,OTGLY,RROnly,4.418196,0.845745,60.19,130.14,190.33,8.12,10.260000,0.26,4.854454
2,ADYEY,RROnly,7.248964,0.867021,143.82,155.70,299.52,12.89,15.300000,0.19,4.793233
55,SE,RROnly,39.728645,0.856383,1331.81,1618.62,2950.43,67.52,76.400002,0.13,3.907529
24,FQVLF,RROnly,6.997321,0.888298,208.71,215.01,423.73,12.84,13.520000,0.05,1.876900
26,GLOB,RROnly,144.057329,0.718085,933.69,1244.74,2178.43,161.12,189.080002,0.17,1.812534
30,HYMTF,RROnly,21.628758,0.856383,193.83,168.37,362.20,56.38,58.630001,0.04,0.000000
1,ADDDF,RROnly,95.817382,0.872340,1226.62,1204.91,2431.54,252.17,257.519989,0.02,0.000000


In [ ]:
findata.EPOCHS=200
tickers6 = ['ARGT', 'EWD', 'FLN', 'FM',
           'ECH', 'EPHE', 'EWA', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'THD', 'TUR', 'VNM', 'XCEM']
mod6 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df6 = pipeline.runModelCombined(tickers6, 'cetf-3b', mod6, True)[0]


In [ ]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
14,EWZ,RROnly,20.870570,0.946809,163.56,134.32,297.88,29.60,31.480000,0.06,8.581560
11,EWP,RROnly,18.915227,0.914894,109.69,65.86,175.56,33.66,29.010000,-0.14,8.541033
6,EWA,RROnly,16.264419,0.936170,94.19,62.48,156.67,24.65,22.930000,-0.07,8.449848
9,EWC,RROnly,23.547080,0.920213,110.88,82.88,193.76,38.26,34.590000,-0.10,8.404255
22,XCEM,RROnly,20.731747,0.930851,109.14,71.15,180.30,30.51,28.850000,-0.05,7.925532
2,FLN,RROnly,12.479329,0.930851,88.30,48.80,137.11,19.31,18.270000,-0.05,7.925532
20,TUR,RROnly,16.397902,0.888298,185.40,124.43,309.83,42.28,37.439999,-0.11,7.867505
7,EWI,RROnly,20.596518,0.877660,122.11,76.85,198.96,38.83,33.860001,-0.13,7.835516
1,EWD,RROnly,26.322633,0.925532,134.10,155.81,289.91,41.93,39.959999,-0.05,7.765957
19,THD,RROnly,56.310138,0.909574,217.91,250.43,468.35,57.55,60.330002,0.05,7.287234


In [ ]:
findata.EPOCHS=200
review = ['AMR', 'IFF', 'BLK']
tickers7 = ['AAL', 'ADI', 'ALB', 'ANF', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL', 'PXD',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TCN', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

mod7 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df7 = pipeline.runModelCombined(tickers7, 'val-5a', mod7, False)[0]

In [8]:
addAlloc(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
54,KVUE,RROnly,17.879241,0.953488,10.19,30.07,40.26,18.39,19.459999,0.06,8.759690
9,BAX,RROnly,31.732772,0.904255,180.04,290.56,470.60,33.16,39.580002,0.19,8.538634
59,NTR,RROnly,29.110441,0.904255,451.17,288.11,739.28,55.41,64.650002,0.17,8.479349
5,AQN,RROnly,4.968251,0.936170,41.59,55.56,97.15,5.94,6.370000,0.07,8.449848
75,SAFE,RROnly,16.065079,0.888298,416.94,575.85,992.78,19.06,22.940001,0.20,8.324468
11,BGFV,RROnly,1.650961,0.882979,108.73,129.18,237.91,3.27,3.900000,0.19,8.213886
66,PETS,RROnly,4.463863,0.861702,77.42,123.24,200.66,4.34,5.500000,0.27,8.104807
67,PNR,RROnly,34.983571,0.914894,300.59,244.97,545.56,77.55,83.169998,0.07,7.933131
2,ALB,RROnly,77.683170,0.882979,1808.97,1708.83,3517.81,114.94,130.119995,0.13,7.929624
61,OLN,RROnly,10.142511,0.888298,325.81,313.54,639.35,50.07,55.500000,0.11,7.867505


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'FXY', 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']
mod8 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df8 = pipeline.runModelCombined(tickers8, 'com-6a', mod8, True)[0]

In [10]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
4,SLV,RROnly,16.525718,0.925532,112.88,90.04,202.92,26.67,28.340000,0.06,8.014184
0,FXY,RROnly,59.183566,0.904255,84.37,165.38,249.75,59.00,63.480000,0.08,7.845745
7,TLT,RROnly,81.844570,0.898936,224.26,330.29,554.56,91.50,85.720001,-0.06,7.304965
1,GLD,RROnly,152.558409,0.914894,491.76,432.03,923.79,211.60,219.740005,0.04,7.021277
10,WEAT,RROnly,5.004930,0.819149,34.19,34.31,68.50,5.90,6.710000,0.14,6.899696
5,SOYB,RROnly,14.039792,0.872340,84.16,61.68,145.84,24.67,25.350000,0.03,4.468085
2,ISHG,RROnly,63.519917,0.920213,65.64,130.73,196.37,69.90,70.540001,0.01,1.223404
6,TIP,RROnly,100.435658,0.867021,98.49,104.66,203.14,106.05,104.790001,-0.01,0.000000
3,PDBC,RROnly,7.095595,0.840426,51.31,21.92,73.23,13.75,13.550000,-0.01,0.000000
8,UUP,RROnly,22.566834,0.856383,37.29,20.80,58.09,28.77,28.379999,-0.01,0.000000


In [ ]:
findata.EPOCHS=200
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
mod9 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df9 = pipeline.runModelCombined(tickers9, 'cry-5a', mod9, True)[0]

In [ ]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
9,LINK-USD,RROnly,5.700525,0.867021,135.15,174.84,309.98,13.48,11.040000,-0.18,5.454238
13,TON-USD,RROnly,1.163133,0.840426,19.67,45.52,65.20,2.26,2.720000,0.20,5.205167
12,SOL-USD,RROnly,14.358925,0.819149,1040.77,1213.17,2253.94,145.15,117.540001,-0.19,4.899216
1,AVAX-USD,RROnly,10.921507,0.829787,424.73,677.98,1102.71,33.69,28.760000,-0.15,4.792300
5,DOT-USD,RROnly,4.179479,0.824468,56.84,160.34,217.18,6.72,7.790000,0.16,4.791983
6,ETH-USD,RROnly,1069.302586,0.845745,16282.49,21859.12,38141.62,2921.02,2639.199951,-0.10,4.498480
10,LTC-USD,RROnly,48.094046,0.787234,747.88,902.75,1650.62,81.05,95.120003,0.17,4.371536
4,DOGE-USD,RROnly,0.058617,0.797872,1.33,1.15,2.47,0.14,0.120000,-0.15,4.351570
0,ADA-USD,RROnly,0.277814,0.840426,3.43,10.87,14.30,0.44,0.480000,0.09,4.229990
7,FIL-USD,RROnly,3.865145,0.728723,60.28,238.98,299.26,5.64,6.960000,0.23,4.025704


In [11]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
com-6a-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 3,220,531 bytes  received 35 bytes  6,441,132.00 bytes/sec
total size is 167,046,464  speedup is 51.87
